In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [54]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "First Up, ask the user to select one of the following languages - 1. English, 2. Spanish, 3. French, 4. Hindi"
system_message += "Prompt this to the user in all four languages - English, Spanish, French and Hindi"

In [55]:
 global selected_language  # Use the global variable to track the language

In [56]:
# function that displays cost of ticket
ticket_prices = {"pune-london": "$799", "pune-paris": "$899", "pune-tokyo": "$1400", "pune-berlin": "$499"}

def get_ticket_price(source_city, destination_city):
    print(f"Tool get_ticket_price called for {source_city} to {destination_city}")
    city = source_city.lower() + "-" + destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [57]:
# function that sets the language for communication
language_options = {"1": "English", "2": "Spanish", "3": "French", "4": "Hindi"}

def choose_language_function(language):
    print(f"Tool choose_language_function called with language {language} ")
    return language_options.get(language, "Unknown")

In [58]:
choose_language_function("2")

Tool choose_language_function called with language 2 


'Spanish'

In [69]:
# function that prints ticket
tickets = {"pune-london": {"from": "Pune", "to": "London", "price" : "$799", "departure_date_time": "25-04-2025 09:05 hrs", "arrival_date_time": "25-04-2025 10:15 hrs"}, 
           "pune-paris":  {"from": "Pune", "to": "Paris", "price" : "$899", "departure_date_time": "25-04-2025 19:05 hrs", "arrival_date_time": "25-04-2025 22:15 hrs"}, 
           "pune-tokyo":  {"from": "Pune", "to": "Tokyo", "price" : "$1400", "departure_date_time": "26-04-2025 21:05 hrs", "arrival_date_time": "26-04-2025 21:15 hrs"}, 
           "pune-berlin":  {"from": "Pune", "to": "Berlin", "price" : "$499", "departure_date_time": "27-04-2025 22:05 hrs", "arrival_date_time": "27-04-2025 18:15 hrs"},
    	   "पुणे-लंदन": {"from": "Pune", "to": "London", "price" : "$799", "departure_date_time": "25-04-2025 09:05 hrs", "arrival_date_time": "25-04-2025 10:15 hrs"}, 
           "पुणे-पेरिस":  {"from": "Pune", "to": "Paris", "price" : "$899", "departure_date_time": "25-04-2025 19:05 hrs", "arrival_date_time": "25-04-2025 22:15 hrs"}, 
           "पुणे-टोकियो":  {"from": "Pune", "to": "Tokyo", "price" : "$1400", "departure_date_time": "26-04-2025 21:05 hrs", "arrival_date_time": "26-04-2025 21:15 hrs"}, 
           "पुणे-बर्लिन":  {"from": "Pune", "to": "Berlin", "price" : "$499", "departure_date_time": "27-04-2025 22:05 hrs", "arrival_date_time": "27-04-2025 18:15 hrs"}}

def get_print_ticket(source_city, destination_city, num_persons):
    print(f"Tool get_print_ticket called for {source_city} to {destination_city}")
    city = source_city.lower() + "-" + destination_city.lower()
    print(city)
    ticket = tickets.get(city, "Unknown")
    return ticket

In [70]:
get_print_ticket("Pune","Tokyo", 5)

Tool get_print_ticket called for Pune to Tokyo
pune-tokyo


{'from': 'Pune',
 'to': 'Tokyo',
 'price': '$1400',
 'departure_date_time': '26-04-2025 21:05 hrs',
 'arrival_date_time': '26-04-2025 21:15 hrs'}

In [71]:
print_function = {
    "name": "get_print_ticket",
    "description": "Print the ticket. Call this whenever you need to print the ticket, for example when a customer asks 'Book the ticket'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "source_city": {
                "type": "string",
                "description": "The city that the customer wants to start travel from",
            },
            "num_persons": {
                "type": "integer",
                "description": "The number of persons travelling",
            },
        },
        "required": ["destination_city", "source_city", "num_persons"],
        "additionalProperties": False
    }
}

In [72]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "source_city": {
                "type": "string",
                "description": "The city that the customer wants to start travel from",
            },
        },
        "required": ["destination_city", "source_city"],
        "additionalProperties": False
    }
}

In [73]:
language_function = {
    "name": "choose_language_function",
    "description": "Read the language of choice from the user, for example when a customer says 'Spanish', it means he/she wants all further communications in Spanish",
    "parameters": {
        "type": "object",
        "properties": {
            "language": {
                "type": "string",
                "description": "The language of communication chosen by the user",
            },
        },
        "required": ["language"],
        "additionalProperties": False
    }
}

In [74]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, 
         {"type": "function", "function": print_function},
         {"type": "function", "function": language_function}]

In [75]:
def chat(message, history):
    if(message == "2"): 
        message = "Use Spanish for all further communications in this chat"
    elif(message == "3"):
        message = "Use French for all further communications in this chat"
    elif(message == "4"):
        message = "Use Hindi for all further communications in this chat"

    print("-------------------------beginning of messages---------------------")
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(messages)
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, source_city, destination_city  = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [76]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    
    arguments = json.loads(tool_call.function.arguments)
    source_city = arguments.get('source_city')
    destination_city = arguments.get('destination_city')
    language = arguments.get('language')

    tool_name = tool_call.function.name

    if tool_name == "get_ticket_price":
        price = get_ticket_price(source_city, destination_city)
        response = {
            "role": "tool",
            "content": json.dumps({"source_city": source_city, "destination_city": destination_city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, source_city, destination_city
    elif tool_name == "choose_language_function":
        language = choose_language_function(arguments.get('language'))
        response = {
            "role": "tool",
            "content": json.dumps({"language": language}),
            "tool_call_id": tool_call.id
        }
        return response, source_city, destination_city
    else:
        num_persons = arguments.get('num_persons')
        ticket_details = get_print_ticket(source_city, destination_city, num_persons)
        response = {
            "role": "tool",
            "content": json.dumps({"source_city": source_city, "destination_city": destination_city,"price": ticket_details}),
            "tool_call_id": tool_call.id
        }
        return response, source_city, destination_city

In [78]:
# Chat logic
def chat_interface(message, history): 
    # Once a language is selected, respond in that language
    return chat(message, history)

gr.ChatInterface(fn=chat_interface, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


-------------------------beginning of messages---------------------
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so.First Up, ask the user to select one of the following languages - 1. English, 2. Spanish, 3. French, 4. HindiPrompt this to the user in all four languages - English, Spanish, French and Hindi"}, {'role': 'user', 'content': 'hi'}]
-------------------------beginning of messages---------------------
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so.First Up, ask the user to select one of the following languages - 1. English, 2. Spanish, 3. French, 4. HindiPrompt this to the user in all four languages - English, Spanish, French and Hindi"}, {'role': 'user', 'metadata': 